# Task 6: Introduction to Video Processing with OpenCV

## Section 1: Setup and Video Loading

### **Task 1**: Load and Display Video
*Instruction*: Load a video file (`sample_video.mp4`) and display it frame-by-frame using OpenCV.

In [2]:
import cv2
from google.colab.patches import cv2_imshow  # Important for Colab!

# Load video
cap = cv2.VideoCapture('sample_video.mp4')  # Make sure your file is uploaded

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Display frame using cv2_imshow for Colab
    cv2_imshow(frame)

    # Wait for 25 ms
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Section 2: Convert Frames to Grayscale

### **Task 2**: Display Grayscale Video Frames

*Instruction*: Convert each frame to grayscale before displaying.


In [ ]:
import cv2
import matplotlib.pyplot as plt

cap = cv2.VideoCapture('sample_video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    plt.imshow(gray, cmap='gray')
    plt.axis('off')
    plt.show()

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Section 3: Save Processed Video

### **Task 3**: Save Grayscale Video to File

*Instruction*: Save the grayscale video to disk as `output_gray.avi`.


In [ ]:
import cv2

# Open the input video
cap = cv2.VideoCapture('sample_video.mp4')  # Replace with your input video filename

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
out = cv2.VideoWriter('output_gray.avi',
                      cv2.VideoWriter_fourcc(*'XVID'),
                      fps,
                      (frame_width, frame_height),
                      isColor=False)  # Set isColor=False for grayscale

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

import cv2

# Open the input video
cap = cv2.VideoCapture('sample_video.mp4')  # Replace with your input video filename

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
out = cv2.VideoWriter('output_gray.avi',
                      cv2.VideoWriter_fourcc(*'XVID'),
                      fps,
                      (frame_width, frame_height),
                      isColor=False)  # Set isColor=False for grayscale

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Write the grayscale frame
    out.write(gray_frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

print("Grayscale video saved as 'output_gray.avi'")

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

print("Grayscale video saved as 'output_gray.avi'")


Grayscale video saved as 'output_gray.avi'
Grayscale video saved as 'output_gray.avi'


## Section 4: Real-Time Webcam Feed

### **Task 4**: Capture and Display Webcam Feed

*Instruction*: Access the webcam and display the live video feed. Press `q` to quit.

In [ ]:
# Install required libraries
!pip install -q pyngrok

# Import libraries
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np

# Function to capture image from webcam
def capture_image():
    js = Javascript('''
        async function captureImage() {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            document.body.appendChild(div);
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture button click
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            video.remove();
            capture.remove();
            div.remove();
            return canvas.toDataURL('image/jpeg', 0.8);
        }
        captureImage();
    ''')

    display(js)
    data = eval_js('captureImage()')
    binary = b64decode(data.split(',')[1])
    np_arr = np.frombuffer(binary, dtype=np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return img

# Now run the capture
print("Click 'Capture' to take a photo 📸")
img = capture_image()


# Show the captured image
import matplotlib.pyplot as plt
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Captured Image')
plt.show()
if cv2.waitKey(1) & 0xFF == ord('q'): # The condition to check if 'q' is pressed
    # Optionally, save the captured image, This code block will only run if 'q' is pressed
    cv2.imwrite('captured_image.jpg', img)
    print("✅ Image saved as captured_image.jpg")

## Section 5: Add Live Effects to Webcam Feed

### **Task 5**:  Apply Canny Edge Detection in Real-Time

*Instruction*: While capturing from webcam, apply Canny edge detection to each frame and display side-by-side.


In [ ]:
# 1. Import necessary libraries
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np

# 2. Function to capture webcam image
def capture_frame():
    js = Javascript('''
    async function captureFrame() {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);
        document.body.appendChild(div);

        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        const video = document.createElement('video');
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit
        video.style.width = '400px';

        div.appendChild(video);

        // Wait for Capture button click
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);

        stream.getTracks().forEach(track => track.stop());
        div.remove();
        return canvas.toDataURL('image/jpeg', 0.8);
    }
    captureFrame();
    ''')
    display(js)
    data = eval_js('captureFrame()')
    binary = b64decode(data.split(',')[1])
    np_arr = np.frombuffer(binary, dtype=np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    return img

# 3. Capture a frame
frame = capture_frame()

# 4. Apply Canny edge detection
edges = cv2.Canny(frame, 100, 200)

# 5. Stack images side-by-side
combined = np.hstack((frame, cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)))

# 6. Show the result
from google.colab.patches import cv2_imshow
cv2_imshow(combined)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Section 6: Color Spaces and Histogram

### **Task 6**: Convert Color Spaces and Plot Histogram

*Instruction*: Convert the image to grayscale and HSV. Then plot a histogram of grayscale values.

In [ ]:
import cv2
import matplotlib.pyplot as plt


image_path = 'sample.jpg'
image = cv2.imread(image_path)


if image is None:
    print(f"Error: Could not load image from {image_path}. Please check the path and ensure the image file exists.")
else:
    #  Convert to Grayscale and HSV
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 3. Plot histogram of grayscale
    plt.hist(gray.ravel(), bins=256, range=[0, 256])
    plt.title("Grayscale Histogram")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.show()